In [11]:
import tensorflow as tf

tf.reset_default_graph()
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters

num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# the lstm cell
lstm = tf.contrib.rnn.BasicLSTMCell(num_hidden)
initial_state = lstm.zero_state(1, dtype=tf.float32)

####convert input to list
input_data_list = tf.transpose(X, [1, 0, 2])
input_data_list = tf.reshape(input_data_list, [-1, num_input])
input_data_list = tf.split(axis=0, num_or_size_splits=timesteps, value=input_data_list)

output_list, state = tf.nn.static_rnn(lstm, input_data_list, dtype=tf.float32)

w = tf.Variable(tf.random_normal([num_hidden, num_classes]))
b = tf.Variable(tf.random_normal([num_classes]))

output_reshaped = tf.reshape(output_list, [timesteps, -1, num_hidden])
output_reshaped = tf.transpose(output_reshaped, [1, 0, 2])
output_reshaped = tf.reshape(output_reshaped, [-1, num_hidden])

output_modified = tf.matmul(output_reshaped, w) + b
last_output = tf.reshape(output_modified, [-1, timesteps, num_classes])

## defining loss function
## use cross entropy of softmax of logits as the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=last_output[:, -1, :], labels=Y, name='loss')
## computes the mean over all the examples in the batch
loss = tf.reduce_mean(entropy)
# + regulation_rate*tf.nn.l2_loss(w)

optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss)

##the prediction we made
preds = tf.nn.softmax(last_output[:, -1, :])
##check how many of them are correct arg maxx is used because Y is one hat
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

init = tf.global_variables_initializer()


# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    total_loss = 0
    total_acc = 0
    for step in range(1, training_steps+1):
    #To implement training  
        ##training batches
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        X_batch = X_batch.reshape((-1, timesteps, num_input))
        
        _, loss_batch, acc_batch = sess.run([optimizer, loss, accuracy], feed_dict={X: X_batch, Y: Y_batch})
        total_loss += loss_batch
        total_acc += acc_batch
        if step % 100 == 0:
            print('step {}, Average loss : {}, Accuracy : {:.6f}'.format(step, total_loss / 100,
                                                                                        total_acc / 100))
            total_loss = 0
            total_acc = 0

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:",
          sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz
step 100, Average loss : 2.0095578944683075, Accuracy : 0.294219
step 200, Average loss : 0.8842377665638924, Accuracy : 0.699375
step 300, Average loss : 0.40342644542455675, Accuracy : 0.869922
step 400, Average loss : 0.285279356315732, Accuracy : 0.908047
step 500, Average loss : 0.22100868113338948, Accuracy : 0.931094
step 600, Average loss : 0.18953429825603962, Accuracy : 0.940391
step 700, Average loss : 0.15145260199904442, Accuracy : 0.953203
step 800, Average loss : 0.14632243230938913, Accuracy : 0.953125
step 900, Average loss : 0.11627853091806173, Accuracy : 0.964844
step 1000, Average loss : 0.11626750459894537, Accuracy : 0.963203
step 1100, Average loss : 0.10803536001592874, Accuracy : 0.966875
step 1200, Average loss : 0.09538207659497858, Accuracy : 